# Food Classifier

> food list : apple, orange, strawberry, cucumber, salmon, egg, kimchi, milk, pizza, broccoli, sausage, carrot

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import EarlyStopping


# dimensions of our images.
img_width, img_height = 225, 225

train_data_dir = r'C:\Users\Jooyoung\Desktop\data1\train'
validation_data_dir = r'C:\Users\Jooyoung\Desktop\data1\validation'
nb_train_samples = 6190
nb_validation_samples = 703
epochs = 70
batch_size = 100

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(13))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
#정규화: 이미지가 0, 1, 2... 255까지 값을 가지는 2차원 배열/0과 255 사이의 값을 0.0과 1.0 사이의 값으로 바꾸기 위함
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# earlystopping ends training when the validation loss stops improving = overfitting 막아주기 위해
earlystop = EarlyStopping(monitor='val_loss', patience=4, verbose=0, mode='auto')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[earlystop])

model.save('fourth_try.h5')

c:\users\jooyoung\miniconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 6180 images belonging to 13 classes.
Found 703 images belonging to 13 classes.
Epoch 1/70
61/61 [==============================] - 1382s 23s/step - loss: 2.1731 - acc: 0.2679 - val_loss: 1.5872 - val_acc: 0.4371
Epoch 2/70
61/61 [==============================] - 1266s 21s/step - loss: 1.3173 - acc: 0.5496 - val_loss: 1.1690 - val_acc: 0.6157
Epoch 3/70
61/61 [==============================] - 933s 15s/step - loss: 1.0100 - acc: 0.6609 - val_loss: 0.9368 - val_acc: 0.6771
Epoch 4/70
61/61 [==============================] - 779s 13s/step - loss: 0.8993 - acc: 0.7047 - val_loss: 0.9769 - val_acc: 0.7000
Epoch 5/70
61/61 [==============================] - 673s 11s/step - loss: 0.7932 - acc: 0.7356 - val_loss: 0.8049 - val_acc: 0.7329
Epoch 6/70
61/61 [==============================] - 695s 11s/step - loss: 0.7337 - acc: 0.7578 - val_loss: 0.8213 - val_acc: 0.7486
Epoch 7/70
61/61 [==============================] - 658s 11s/step - loss: 0.6574 - acc: 0.7821 - val_loss: 0.8047 - val_a

In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 223, 223, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 223, 223, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 109, 109, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 52, 52, 64)        18496     
__________

# 안드로이드용 Freeze Graph

In [4]:
# This was created with @warptime's help. Thank you!

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras.models import load_model
from keras import backend as K
import os.path as osp
import os
import tensorflow as tf

model = load_model('fourth_try.h5')
nb_classes = 1 # The number of output nodes in the model
prefix_output_node_names_of_final_network = 'output_node'

K.set_learning_phase(0)

pred = [None]*nb_classes
pred_node_names = [None]*nb_classes
for i in range(nb_classes):
    pred_node_names[i] = prefix_output_node_names_of_final_network+str(i)
    pred[i] = tf.identity(model.output[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)

sess = K.get_session()
output_fld = 'tensorflow_model/'
if not os.path.isdir(output_fld):
    os.mkdir(output_fld)
output_graph_name = 'No_Answer_model' + '.pb'
output_graph_suffix = '_inference'

constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
graph_io.write_graph(constant_graph, output_fld, output_graph_name, as_text=False)
print('saved the constant graph (ready for inference) at: ', osp.join(output_fld, output_graph_name))

output nodes names are:  ['output_node0']
INFO:tensorflow:Froze 10 variables.
Converted 10 variables to const ops.
saved the constant graph (ready for inference) at:  tensorflow_model/No_Answer_model.pb


# input/output node 이름 확인하기

In [9]:
def print_graph_nodes(filename):

    import tensorflow as tf

    g = tf.GraphDef()

    g.ParseFromString(open(filename, 'rb').read())

    print()

    print(filename)

    print("=======================INPUT=========================")

    print([n for n in g.node if n.name.find('input') != -1])

    print("=======================OUTPUT========================")

    print([n for n in g.node if n.name.find('output') != -1])

    print("===================KERAS_LEARNING=====================")

    print([n for n in g.node if n.name.find('keras_learning_phase') != -1])

    print("======================================================")

    print()
    

pbfile= r'C:\Users\Jooyoung\tensorflow_model\No_Answer_model.pb'
print_graph_nodes(pbfile)


C:\Users\Jooyoung\tensorflow_model\No_Answer_model.pb
=======================INPUT=========================
[name: "conv2d_1_input_2"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 225
      }
      dim {
        size: 225
      }
      dim {
        size: 3
      }
    }
  }
}
]
=======================OUTPUT========================
[name: "output_node0"
op: "Identity"
input: "strided_slice"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
]
===================KERAS_LEARNING=====================
[]



# evaluation&prediction

In [18]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras.models import load_model
from keras import backend as K
import os.path as osp
import os
import tensorflow as tf

tf.set_random_seed(777)
np.random.seed(777)

# dimensions of our images.
img_width, img_height = 150, 150

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_data_dir = r'C:\Users\Jooyoung\Desktop\data1\train'
validation_data_dir = r'C:\Users\Jooyoung\Desktop\data1\validation'
nb_train_samples = 6190
nb_validation_samples = 703
epochs = 70
batch_size = 100

test_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


model = load_model('second_try.h5')

# 6. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(validation_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 7. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(validation_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(validation_generator.class_indices)
print(output)

Found 703 images belonging to 13 classes.
-- Evaluate --
acc: 88.00%
-- Predict --
{'apple': 0, 'broccoli': 1, 'carrot': 2, 'cucumber': 3, 'egg': 4, 'kimchi': 5, 'milk': 6, 'onion': 7, 'orange': 8, 'pizza': 9, 'salmon': 10, 'sausage': 11, 'strawberry': 12}
[[0.000 0.000 0.000 ... 0.000 0.000 0.000]
 [0.000 0.000 0.000 ... 0.292 0.000 0.000]
 [0.000 0.000 0.000 ... 0.000 0.000 0.000]
 ...
 [0.000 0.000 0.000 ... 0.000 0.000 0.000]
 [0.000 0.000 0.000 ... 0.000 0.000 0.000]
 [0.000 0.000 0.000 ... 0.000 0.000 0.000]]


In [13]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras.models import load_model
from keras import backend as K
import os.path as osp
import os
import tensorflow as tf

tf.set_random_seed(777)
np.random.seed(777)

# dimensions of our images.
img_width, img_height = 225, 225

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_data_dir = r'C:\Users\Jooyoung\Desktop\data1\train'
validation_data_dir = r'C:\Users\Jooyoung\Desktop\data1\validation2'
nb_train_samples = 6190
nb_validation_samples = 26
epochs = 70
batch_size = nb_validation_samples

test_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

model = load_model('fourth_try.h5')

# 7. 모델 사용하기
print("-- Predict --")
#filenames = validation_generator.filenames
#i = len(filenames)
output = model.predict_generator(validation_generator, steps=1)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(validation_generator.class_indices)
#print(validation_generator.filenames)
print(output)

Found 26 images belonging to 13 classes.
-- Predict --
{'apple': 0, 'broccoli': 1, 'carrot': 2, 'cucumber': 3, 'egg': 4, 'kimchi': 5, 'milk': 6, 'onion': 7, 'orange': 8, 'pizza': 9, 'salmon': 10, 'sausage': 11, 'strawberry': 12}
[[0.998 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
  0.002]
 [0.988 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.001
  0.011]
 [0.001 0.568 0.003 0.003 0.003 0.000 0.001 0.000 0.000 0.000 0.000 0.416
  0.003]
 [0.000 0.999 0.001 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
  0.000]
 [0.000 0.000 0.884 0.000 0.000 0.002 0.000 0.018 0.003 0.000 0.092 0.001
  0.000]
 [0.000 0.000 0.999 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
  0.000]
 [0.010 0.013 0.001 0.969 0.000 0.000 0.004 0.000 0.001 0.000 0.000 0.002
  0.000]
 [0.027 0.013 0.011 0.935 0.000 0.000 0.005 0.003 0.001 0.000 0.002 0.001
  0.000]
 [0.000 0.000 0.000 0.000 0.919 0.000 0.000 0.074 0.000 0.000 0.000 0.007
  0.000]
 [0.002 0.000 0.005 0.00